In [3]:
import requests
import json
response_API = requests.get('https://prod-21.southeastasia.logic.azure.com/workflows/47b0881b3602487cad1385ccc86be53a/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=mOXZz0H8jR9VVlGj_PjqPqHVVdl26gPQ45mWBqyW9MQ')
#print(response_API.status_code)
data = response_API.text
parse_json = json.loads(data)
Data_frame_1 = parse_json['Inputs']['web_service_input']
Data_frame_2 = parse_json['Inputs']['input1'] #[0]["horizon"]

print(Data_frame_1)
print("============================================================")
print(Data_frame_2)

[{'Item': '15-0019138', 'DateKey': '2023-01-01', 'TransactionQty': 330.5}, {'Item': '15-0019138', 'DateKey': '2023-02-01', 'TransactionQty': 331}, {'Item': '15-0019138', 'DateKey': '2023-03-01', 'TransactionQty': 331}]
[{'horizon': 12, 'seasonality': 12, 'forecast_start_datekey': '2023-04-01', 'time_series_model': 'ARIMA', 'confidence_level': 95}]


In [4]:
import json
import pandas as pd

web_service_input_data = parse_json['Inputs']['web_service_input']

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(web_service_input_data)

# Save the DataFrame to a CSV file
#df.to_csv('web_service_input.csv', index=False)

# Print the DataFrame
#print("Saved CSV:")
print(df)

         Item     DateKey  TransactionQty
0  15-0019138  2023-01-01           330.5
1  15-0019138  2023-02-01           331.0
2  15-0019138  2023-03-01           331.0


In [5]:
filtered_df1 = df.drop(['Item'], axis=1)
filtered_df1

,DateKey,TransactionQty
0,2023-01-01,330.5
1,2023-02-01,331.0
2,2023-03-01,331.0


In [1]:
filtered_df1['DateKey'] =pd.to_datetime(filtered_df1['DateKey'])
filtered_df2 = filtered_df1[['DateKey','TransactionQty']]
filtered_df1.info()

NameError: name 'pd' is not defined

In [7]:
filtered_df1.set_index('DateKey', inplace=True)
filtered_df1

,TransactionQty
DateKey,
2023-01-01,330.5
2023-02-01,331.0
2023-03-01,331.0


In [9]:
pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 116.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [statsmodels] [statsmodels]
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import joblib

# Load your time series data
# Assume you have a pandas DataFrame with a column 'value' representing your time series
# Replace this with your actual data loading code
#data = pd.read_csv("your_time_series_data.csv")

# Train ARIMA model
model = ARIMA(filtered_df1['TransactionQty'], order=(5, 1, 0))  # Example order, tune this based on your data
fitted_model = model.fit()

# Save the trained model
joblib.dump(fitted_model, "arima_model.pkl")

NameError: name 'filtered_df1' is not defined

In [16]:
pip install azureml.core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.0/608.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37/37 [azureml.core] [azureml.core]thorization]try]
Note: you may need to restart the kernel to use updated packages.


In [3]:
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Register the model

model = Model.register(model_path="arima_model.pkl",
                       model_name="my_model",
                       workspace=ws)


# Create an environment
env = Environment(name="my_env")
cd = CondaDependencies.create(pip_packages=["numpy", "pandas", "scikit-learn",'joblib','azureml-defaults','inference-schema[numpy-support]','statsmodels','pmdarima','tbats','stldecompose'])
env.python.conda_dependencies = cd

# Create an inference configuration
inference_config = InferenceConfig(entry_script="score_3.py",
                                   environment=env)

# Deploy the model as a web service
deployment_config = AciWebservice.deploy_configuration(cpu_cores=0.5,
                                                         memory_gb=0.5)


print('Name:', model.name)
print('Version:', model.version)

Registering model my_model
Name: my_model
Version: 7


In [4]:
#Deploy the model

service = Model.deploy(workspace=ws,
                       name="salse-forecasting-corection-4",
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_3415/3112442987.py:3: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-07-04 07:19:48+00:00 Creating Container Registry if not exists.
2025-07-04 07:19:48+00:00 Registering the environment.
2025-07-04 07:19:51+00:00 Use the existing image.
2025-07-04 07:19:52+00:00 Generating deployment configuration.
2025-07-04 07:19:54+00:00 Submitting deployment to compute.
2025-07-04 07:20:00+00:00 Checking the status of deployment salse-forecasting-corection-4..
2025-07-04 07:21:46+00:00 Checking the status of inference endpoint salse-forecasting-corection-4.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [4]:
package = Model.package(ws, [model], inference_config)
package.wait_for_creation(show_output=True)

======Starting Image Build on Compute======
The run ID for the image build on compute is imgbldrun_723f005
Additional logs for the run: https://ml.azure.com/experiments/id/prepare_image/runs/imgbldrun_723f005?wsid=/subscriptions/5fb473f2-c421-40c7-b744-408ca0bcd190/resourcegroups/ML-Test/workspaces/BIT-ML-1&tid=75dd4cdf-4a7b-4318-a629-86ca4a2ecc82
2025-08-29T05:02:05: Logging into Docker registry: ae451621dda14e0b9de742674e7c94f3.azurecr.io
2025-08-29T05:02:05: WARNING! Using --password via the CLI is insecure. Use --password-stdin.

2025-08-29T05:02:05: Login Succeeded
2025-08-29T05:02:05: WARNING! Your credentials are stored unencrypted in '/root/.docker/config.json'.
2025-08-29T05:02:05: Configure a credential helper to remove this warning. See
2025-08-29T05:02:05: https://docs.docker.com/go/credential-store/



2025-08-29T05:02:05: Logging into Docker registry: ae451621dda14e0b9de742674e7c94f3.azurecr.io
2025-08-29T05:02:05: WARNING! Using --password via the CLI is insecure. Use --

In [5]:
package.pull()

Pulling image from ACR (this may take a few minutes depending on image size)...

ae451621dda14e0b9de742674e7c94f3.azurecr.io/package@sha256:808b58e604af5e23971b8111401625d3d83849582b7936e26591d99a1876658f: Pulling from package
e1a89dea01a6: Pulling fs layer
f1dec01e9462: Pulling fs layer
9e8d1a2d892f: Pulling fs layer
1901e9167677: Pulling fs layer
7587f6cbfa23: Pulling fs layer
ba174cf3dfbc: Pulling fs layer
01a4dea35e92: Pulling fs layer
1abef6e6d6c3: Pulling fs layer
52acf9834f0a: Pulling fs layer
6fba9b744657: Pulling fs layer
5798c0d3df08: Pulling fs layer
4f4fb700ef54: Pulling fs layer
451bbb0841c9: Pulling fs layer
656120be27a4: Pulling fs layer
5f86e4147273: Pulling fs layer
30f1a7db9a1b: Pulling fs layer
5d8f1d1e1be9: Pulling fs layer
336727f0437f: Pulling fs layer
94b5c23dec77: Pulling fs layer
53fdf48e3c78: Pulling fs layer
01109f5a314a: Pulling fs layer
967fca46f7bd: Pulling fs layer
451bbb0841c9: Waiting
656120be27a4: Waiting
336727f0437f: Waiting
5f86e4147273: Waiting
94b